In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import mne
from tqdm import tqdm
# from blab.itertools import true_meshgrid
import matplotlib
import seaborn as sns
import pingouin as pg
matplotlib.rcParams['pdf.fonttype'] = 42

In [ ]:
d2 = pd.read_csv('../data/baseline_level/pow_avg_D2_base1_base2_f0=6Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr_PrecuneusR_maxt.txt', sep='  ', header=None).iloc[:15]
# d2.insert(loc=9, value=[0, 0])
# d2.insert(loc=9, column=1)
d3 = pd.read_csv('../data/baseline_level/pow_avg_D3_base1_base2_f0=6Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr_PrecuneusR_maxt.txt', sep='  ', header=None)

def gather_df(csv_names):
    d2 = pd.read_csv(csv_names[0], sep='  ', header=None).iloc[:15]
    d3 = pd.read_csv(csv_names[1], sep='  ', header=None).iloc[:15]
    names = ['Day', 'Subject', 'Session']

    index2 = pd.MultiIndex.from_product([['TMS'], np.arange(1, 16, dtype=int), ['Im1', 'Im2']], names=names)
    index3 = pd.MultiIndex.from_product([['Sham'], np.arange(16, 16 + 15, dtype=int), ['Im1', 'Im2']], names=names)

    d2 = d2.stack()
    d3 = d3.stack()

    d2.index = index2
    d3.index = index3

    df = pd.concat([d2, d3]).reset_index()
    df.columns = names + ['Power']
    df = df.query("Subject != 10")
    return df

precuneus_df = gather_df([
    '../data/baseline_level/pow_avg_D2_base1_base2_f0=6Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr_PrecuneusR_maxt.txt',
    '../data/baseline_level/pow_avg_D3_base1_base2_f0=6Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr_PrecuneusR_maxt.txt'
])

front_df = gather_df([
    '../data/baseline_level/pow_avg_D2_base1_base2_f0=10Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr_Front.txt',
    '../data/baseline_level/pow_avg_sham_base1_base2_f0=10Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr.txt'
])

# arr = np.concatenate([d2.to_numpy().flatten(), d3.to_numpy().flatten()]) 

# df = pd.DataFrame(arr, columns=['Pow'], index=index).reset_index()

In [ ]:
front_df.to_csv('../data/src_baseline_dfs/alpha_frontal.csv', index=False)
precuneus_df.to_csv('../data/src_baseline_dfs/theta_precuneusr_maxt.csv', index=False)

In [ ]:
pg.mixed_anova(df, 'Pow', within='Session', between='Day', subject='Subject', correction=True)

In [ ]:
ax = sns.pointplot(x='Session', y='Power', hue='Day', data=df.query('Subject != 10'), dodge=0.05, capsize=0.05, ci=66.666)
ax.figure.savefig('src_pow_4-8Hz-precuneusr-maxt_plot.pdf')

In [ ]:
sas = df['Pow'].to_numpy().reshape(2, 15, 2)
sas = np.swapaxes(sas, 1, 2)
sas[0, :, 9] *= np.nan
sos = sas[0, 0]
sos = sos
sos = [stats.shapiro(i[~np.isnan(i)]).pvalue for i in sas.reshape(4, -1)]
for i, p in zip(['D2 Im1', 'D2 Im2', 'Sham Im1', 'Sham Im2'], sos):
    print(f'{i} pvalue = {p:<0.3f}')

In [ ]:
sns.violinplot(x='Session', y='Pow', hue='Day', data=df.query('Subject != 10 and Subject != 8'))


In [ ]:
a = df.set_index(['Day', 'Subject', 'Session']).unstack()
a = a['Pow', 'Im2'] - a['Pow', 'Im1']
a = pd.DataFrame(a, columns=['SRC_Alpha_Frontal'])
a.to_csv('src_dpow_avg_D2_D3_Im2-Im1_f0=10Hz_df=2Hz_eloreta_Norm_to_Im1_Bgr_Front.txt')

In [ ]:
df.to_csv('theta_frontal_src.csv', index=False)

In [ ]:
sas[1]

In [ ]:
stats.ttest_ind(*sas[0][:, ~(np.isnan(sas[0]).any(0))])

In [ ]:
df.query('Day == "D3" and Session == "Im2"')['Pow'].to_numpy()

In [ ]:
front_df = pd.read_csv('../data/src_baseline_dfs/alpha_frontal.csv')
precuneus_df = pd.read_csv('../data/src_baseline_dfs/theta_precuneusr_maxt.csv')
front_df.set_index(['Day', 'Session', 'Subject'], inplace=True)
precuneus_df.set_index(['Day', 'Session', 'Subject'], inplace=True)
front_df.columns = ['AlphaFrontal']
front_df['ThetaPrecuneus'] = precuneus_df['Power']
front_df.to_csv('tables/BASELINE_BGR-NORM_ELORETA_MAXT_SRC-POWER.csv')